# Groq + Tavily MCP: Real-Time Search
This notebook is designed for Python developers who want to empower Groq inference with real-time web access, enabling Groq models to utilize up-to-date information as context via Tavily's Model Context Protocol (MCP) server that will give models access to various tools. Live web information is critical for AI agents tasked with performing research, monitoring trends, or generating context-aware responses. 

We will achieve this through three simple steps:
1. Set up **Groq MCP client** for fast inference.
2. Set up **Tavily MCP server** for web search, web scraping, and web crawling.
3. Seemlessly **connect the client to the server** through the Responses API.

---

## Getting Started

Follow these steps to set up:
1. **Sign up** for Tavily at [app.tavily.com](https://app.tavily.com/home/) to get your free API key.
2. **Sign up** for Groq at [console.groq.com](https://console.groq.com/keys) to get your free API key.
3. **Copy your API keys** from your Tavily and Groq account dashboards.
4. **Paste your API keys** into the cell below and run the cell.

In [ ]:
# To export your API keys into a .env file, run the following cell (replace with your actual keys):
!echo "TAVILY_API_KEY=<your-tavily-api-key>" >> .env
!echo "GROQ_API_KEY=<your-groq-api-key>" >> .env

In [ ]:
import json
import os
import time
from datetime import datetime

# API Configuration
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Check if API key is set
if not GROQ_API_KEY:
    print("Please set your Groq production API key")
else:
    print("Groq API key configured successfully!")
if not TAVILY_API_KEY:
    print("Please set your Tavily API key")
else:
    print("Tavily API key configured successfully!")

Groq API key configured successfully!
Tavily API key configured successfully!


Select the foundation model to power inference. Let's try OpenAI's flagship open-weight MoE model, [gpt-oss-120b](https://console.groq.com/docs/model/openai/gpt-oss-120b), available via Groq for fast inference.

In [3]:
# Model configuration
MODEL = "openai/gpt-oss-120b"

## Step 1: Set up the Groq client

In [ ]:
from openai import OpenAI

# set up Groq MCP client
client = OpenAI(base_url="https://api.groq.com/api/openai/v1", api_key="GROQ_API_KEY")

## Step 2: Set up Tavily's remote MCP server

In [13]:
# set up Tavily MCP server
tools = [
    {
        "type": "mcp",
        "server_url": f"https://mcp.tavily.com/mcp/?tavilyApiKey={TAVILY_API_KEY}",
        "server_label": "tavily",
        "require_approval": "never",
    }
]

## Step 3: Connect Groq to the Tavily MCP through Groq's OpenAI-compatible Responses API

In [14]:
def connect_groq_to_tavily(client, tools, query):
    """
    Connect Groq client to Tavily server through responses API.

    This function demonstrates the speed and accuracy of combining:
    - Groq's fast LLM inference (500+ tokens/second)
    - Tavily MCP server for real-time web retrieval
    """

    start_time = time.time()

    # Call Groq with Tavily MCP integration using responses API
    response = client.responses.create(
        model=MODEL,
        input=query,
        tools=tools,
        stream=False,
        temperature=0.1,
        top_p=0.4,
    )

    total_time = time.time() - start_time

    # Get response content from responses API
    content = (
        response.output_text if hasattr(response, "output_text") else str(response)
    )

    # collect executed tools (MCP tool calls)
    executed_tools = []

    # Extract MCP calls from response if available
    if hasattr(response, "output") and response.output:
        for output_item in response.output:
            if hasattr(output_item, "type") and output_item.type == "mcp_call":
                executed_tools.append(
                    {
                        "type": "mcp",
                        "arguments": getattr(output_item, "arguments", "{}"),
                        "output": getattr(output_item, "output", ""),
                        "name": getattr(output_item, "name", ""),
                        "server_label": getattr(output_item, "server_label", ""),
                    }
                )
    return {
        "content": content,
        "total_time": total_time,
        "mcp_calls_performed": executed_tools,
        "timestamp": datetime.now().isoformat(),
    }

Let's implement a helper function to display MCP tool calls and their results. This will provide transparency into which tools were called, their arguments, and outputs.

In [15]:
def print_mcp_calls(mcp_calls):
    executed_tools = mcp_calls["mcp_calls_performed"]
    if executed_tools:
        print(f"\nTAVILY MCP CALLS: Found {len(executed_tools)} tool call(s):")
        print("-" * 50)
        for i, tool in enumerate(executed_tools, 1):
            print(f"\nTool Call #{i}")
            print(f"   Type: {tool['type']}")
            print(f"   Tool Name: {tool['name']}")
            print(f"   Server: {tool['server_label']}")
            try:
                if tool["arguments"]:
                    args = (
                        json.loads(tool["arguments"])
                        if isinstance(tool["arguments"], str)
                        else tool["arguments"]
                    )
                    print(f"   Arguments: {args}")

                # Print model results for transparency
                if tool["output"]:
                    output_data = (
                        json.loads(tool["output"])
                        if isinstance(tool["output"], str)
                        else tool["output"]
                    )
                    if isinstance(output_data, dict) and "models" in output_data:
                        print(f"   Models found: {len(output_data['models'])}")
                        for j, model in enumerate(
                            output_data["models"][:5], 1
                        ):  # Show top 5
                            model_name = model.get("id", model.get("name", "Unknown"))
                            print(f"      {j}. {model_name}")
                        if len(output_data["models"]) > 5:
                            print(
                                f"      ... and {len(output_data['models']) - 5} more models"
                            )
                    else:
                        print(f"   Output: {str(output_data)[:200]}...")
            except Exception as e:
                print(f"   Could not parse tool data: {e}")
    print(f"   Total time: {mcp_calls['total_time']:.2f} seconds")
    print(f"   Tavily MCP calls: {len(mcp_calls['mcp_calls_performed'])}")

# Examples

**Note:** Some queries may consume more tokens than others depending on the amount of tool calls the model makes. Please be aware of various rate limits that are tied to your API keys if you happen to run into any rate limit errors. 

---

## Demo 1: AI startup funding announcements

In [16]:
from IPython.display import Markdown

ai_funding_news = connect_groq_to_tavily(
    client,
    tools,
    "What are some recent AI startup funding announcements?",
)

Let's display the agent's response in markdown format.

In [17]:
Markdown(ai_funding_news["content"])

**Recent AI‑startup funding announcements (September 2025 – early October 2025)**  

| Date (2025) | Startup | Funding round & amount | Valuation (if disclosed) | Lead / notable investors | Key focus |
|-------------|---------|------------------------|--------------------------|--------------------------|-----------|
| **Sep 10** | **Cognition AI** (U.S.) | $400 M Series B | $10.2 B | Founders Fund (lead) – also Lux Capital, 8VC, Neo, Elad Gil, Bain Cap Ventures | AI‑coding assistant “Devin” for software development |
| **Sep 10** | **Mistral AI** (France) | €1.7 B (~$2 B) Series C | $13.7 B | ASML Holding (lead) – also Nvidia, General Catalyst, Index Ventures | Open‑source generative‑AI models |
| **Sep 10** | **Runware** (San Francisco) | $13 M Seed | — | Insight Partners (lead) – Andreessen Horowitz Speedrun Fund, Begin Capital, Zero Prime | Unified API for AI‑generated images, video, audio, text |
| **Sep 10** | **Astrus** (Toronto/Waterloo) | $8 M Seed | — | — | Physics‑aware foundation model for analog chip design |
| **Sep 10** | **PixVerse AI** (Beijing) | $60 M Series B | — | Alibaba (lead) – Antler | AI‑powered video creation platform (100 M+ users) |
| **Sep 10** | **Nuclearn** (U.S.) | $10.5 M Series A | — | Blue Bear Capital (lead) – AZ‑VC, Nucleation Capital, SJF Ventures | AI for nuclear‑reactor operations & predictive maintenance |
| **Sep 2** | **Predoc** (NY) | $30 M Series A | — | Base10 Partners (lead) – Northzone, ENIAC Ventures, ERA Ventures | AI‑native platform automating health‑information workflows |
| **Sep 2** | **CHARM Therapeutics** (UK) | $80 M Series B | — | NEA & SR One (lead) – OrbiMed, F‑Prime, Khosla Ventures, NVIDIA | AI‑enabled oncology drug (menin inhibitor) |
| **Sep 2** | **Leo AI** (Massachusetts) | $9.7 M Seed | — | Flint Capital (lead) – a16z scout, Tech Aviv, Two Lanterns VC, Google VP Yossi Matias | Mechanical‑engineering “copilot” (large‑mechanical‑model) |
| **Sep 2** | **Camera Intelligence** (London) | €1.7 M Seed | — | Betaworks, F4 Fund, Next Wave (Flybridge), 7pc Ventures, Digital Catapult | AI‑integrated camera with LLM‑based voice assistant |
| **Sep 1** | **LayerX** (Japan) | $100 M Series B | — | Technology Cross Ventures (TCV, lead) – MUFG Bank, JAFCO, Keyrock Capital | AI SaaS for enterprise back‑office automation |
| **Sep 16** | **Figure** (San Jose) – AI robotics | $1 B committed capital (Series C) | $39 B post‑money | Parkway Venture Capital (lead) – Nvidia, Intel Capital, LG Tech Ventures, Salesforce, T‑Mobile Ventures, Qualcomm Ventures, Brookfield Asset Mgmt, Macquarie Capital | Humanoid robots for logistics, manufacturing & home use |
| **Oct 12** | **EliseAI** (NY) | $250 M Series E | $2.2 B | Andreessen Horowitz (lead) – Bessemer, Sapphire Ventures | Vertical AI for real‑estate & healthcare automation |
| **Oct 12** | **Cohere** (Toronto) | $500 M Series D | $6.8 B | Radical Ventures & Inovia Capital (co‑lead) – AMD Ventures, Nvidia, Salesforce Ventures | Enterprise‑focused generative AI (sovereign AI) |
| **Oct 12** | **Perplexity AI** (U.S.) | $500 M Series D | $9 B | — (investors not disclosed) | AI‑powered search with citation‑rich answers |
| **Oct 12** | **Databricks** (U.S.) | $10 B Series G | $62 B | Thrive Capital (lead) – Andreessen Horowitz, DST Global, GIC, Insight Partners, WCM Investment Mgmt | AI‑enabled data‑analytics platform |
| **Oct 12** | **OpenAI** (U.S.) | $8.3 B (new round) | $300 B (potential) | SoftBank (lead, $15‑25 B) – others | Scaling foundational models, safety research, enterprise tools |
| **Oct 12** | **Anthropic** (U.S.) | $5 B (potential) | $170 B (rumored) | Lightspeed Ventures (lead) – Amazon (existing) | Claude agentic‑AI system |
| **Oct 12** | **Thinking Machines Lab** (U.S.) | $2 B Series B | $10 B | DST Global, Sequoia, General Catalyst (lead) | Agentic‑AI infrastructure & foundation models |
| **Oct 12** | **Glean** (U.S.) | $150 M Series F | $7.2 B | Kleiner Perkins, Lightspeed, Sequoia, Coatue, General Catalyst | Enterprise AI agents that surface organizational knowledge |
| **Oct 12** | **Lambda Labs** (U.S.) | $480 M Series D | — | — (undisclosed) | GPU cloud for AI training & inference |
| **Oct 12** | **Cast AI** (Lithuania) | $108 M Series C | $850 M | Vintage Investment, Cota Capital, Creandum | AI‑driven cloud‑cost optimization |
| **Oct 12** | **Nexthop AI** (U.S.) | $110 M Series A | — | Lightspeed Ventures (lead) – Kleiner Perkins, WestBridge, Battery Ventures | AI‑focused networking hardware for hyperscalers |
| **Oct 12** | **Valid** (San Francisco) | $5.5 M Seed | — | Canaan Partners | AI‑powered digital‑ad agency automating campaign optimization |
| **Oct 12** | **iEduGPT** (Singapore) | $10 M Seed | — | New Wheel Capital, Baywise, SWC Global, K3 Ventures, Welight | Adaptive AI for professional & academic exam prep |
| **Oct 12** | **Arise AI** (U.S.) | $100 M Series B | — | Tiger Global (lead) – Samsung Ventures, HH‑CTBC, In‑Q‑Tel, RTX Ventures | Analog in‑memory AI accelerator chips |
| **Oct 12** | **Latent Labs** (U.S.) | $50 M Series B | — | Radical Ventures & Sofinnova Partners (co‑lead) | AI‑driven protein‑design platform (ex‑DeepMind AlphaFold) |
| **Oct 12** | **CUSP AI** (U.S.) | $100 M Series A | — | Index Ventures, Lightspeed Ventures | AI‑driven materials discovery for climate‑tech |
| **Oct 12** | **Mistral AI** (France) – additional raise | Targeting $1 B at $10 B valuation (ongoing) | — | MGX (Abu Dhabi) & others (rumored) | Scaling open‑source LLMs & data‑center build‑out |

### Take‑aways
* **AI‑related capital is surging:** $118 B raised in the first half of 2025 (≈ 75 % YoY growth), with AI accounting for ~48 % of all global VC money.
* **Concentration on a few mega‑rounds:** Eight companies (including OpenAI, Figure, Mistral, Cohere, Databricks, Anthropic, Anduril, and Safe Superintelligence) captured ~62 % of AI funding.
* **Investor mix:** Large‑cap tech corporates (Nvidia, Google, Meta, Salesforce, SpaceX), sovereign‑wealth/PE funds (SoftBank, Greenoaks, Brookfield), and top‑tier VCs (Lightspeed, Andreessen Horowitz, Sequoia, Khosla, Accel) are leading the biggest checks.
* **Sector breadth:** Funding spans **foundational models**, **AI‑powered developer tools**, **enterprise AI agents**, **AI‑chip & robotics**, **AI‑generated media**, **health‑tech**, **fin‑tech**, **materials & climate‑tech**, and **AI‑driven legal/education platforms**.

These announcements illustrate the breadth of the AI boom—both deep‑tech (hardware, chips, robotics) and application‑layer startups are attracting multi‑hundred‑million to multi‑billion‑dollar rounds, pushing valuations into the tens‑of‑billions.

Let's examine the agent's intermediate steps, including how it calls different tools and configures tool arguments such as `search_depth`, `time_range`, `max_results`, and more.

In [18]:
print_mcp_calls(ai_funding_news)


TAVILY MCP CALLS: Found 2 tool call(s):
--------------------------------------------------

Tool Call #1
   Type: mcp
   Tool Name: tavily_search
   Server: tavily
   Arguments: {'max_results': 10, 'query': 'AI startup funding announcements September 2025', 'search_depth': 'advanced', 'time_range': 'month'}
   Output: {'query': 'AI startup funding announcements September 2025', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://intellizence.com/insights/startup-funding/weekly-to...

Tool Call #2
   Type: mcp
   Tool Name: tavily_extract
   Server: tavily
   Arguments: {'format': 'markdown', 'urls': ['https://theaiworld.org/news/ai-startups-secure-major-funding-in-september-2025-signaling-global-innovation-surge', 'https://www.crescendo.ai/news/latest-vc-investment-deals-in-ai-startups', 'https://news.crunchbase.com/venture/big-dollar-ai-investors-2025-softbank/', 'https://www.reuters.com/business/robotics-startup-figure-valued-39-billion-latest-fun

## Demo 2: Product Search


In [20]:
product_search = connect_groq_to_tavily(
    client,
    tools,
    "Find the iphone models currently available on apple.com",
)

Let's display the agent's response in markdown format.

In [21]:
Markdown(product_search["content"])

**iPhone models that are currently available for purchase on Apple .com (global store – as of Sept 2025)**  

| Series | Models that can be bought today | Key notes |
|--------|--------------------------------|-----------|
| **iPhone 16 series** | • iPhone 16 Pro  <br>• iPhone 16 Pro Max  <br>• iPhone 16  <br>• iPhone 16 Plus  <br>• iPhone 16e (the “iPhone Air”) | The newest lineup launched in 2025. All five models appear on Apple’s *Compare iPhone* page with active **Buy** links 【0†L1-L9】. |
| **iPhone 15 series** | • iPhone 15 Pro  <br>• iPhone 15 Pro Max  <br>• iPhone 15  <br>• iPhone 15 Plus | Still sold alongside the 16 series; they are listed on the same comparison page and have “Buy” buttons 【0†L1-L9】. |
| **iPhone 14 series** | • iPhone 14 Pro  <br>• iPhone 14 Pro Max  <br>• iPhone 14  <br>• iPhone 14 Plus | Part of Apple’s current catalog (shown in the full model list on the compare page) 【0†L1-L9】. |
| **iPhone 13 series** | • iPhone 13 Pro  <br>• iPhone 13 Pro Max  <br>• iPhone 13 mini  <br>• iPhone 13 | Still listed on the comparison page, indicating they remain available for purchase 【0†L1-L9】. |
| **iPhone SE** | • iPhone SE (3rd generation) | The latest SE model is included in the comparison list and is sold on Apple.com 【0†L1-L9】. |

> **What the source shows**  
> The Apple “Compare iPhone models” page (e.g., https://www.apple.com/in/iphone/compare/) lists **all** iPhone models that Apple currently sells. The page’s model list (excerpted below) includes the nine entries above, plus older generations that are shown for comparison only. The presence of active “Buy” links for the 16‑, 15‑, 14‑, 13‑series and SE (3rd gen) confirms that these are the models you can actually order today.  

*Excerpt from the compare page (showing the model list):*  

```
iPhone 16 Pro
iPhone 16 Pro Max
iPhone 16
iPhone 16 Plus
iPhone 16e
iPhone 15 Pro
iPhone 15 Pro Max
iPhone 15
iPhone 15 Plus
iPhone 14 Pro
iPhone 14 Pro Max
iPhone 14
iPhone 14 Plus
iPhone 13 Pro
iPhone 13 Pro Max
iPhone 13 mini
iPhone 13
iPhone SE (3rd generation)
… (older models follow)
```  

**In short:** the current purchasable iPhone lineup on Apple’s website consists of the iPhone 16 family, the iPhone 15 family, the iPhone 14 family, the iPhone 13 family, and the latest iPhone SE (3rd gen).

Let's examine the agent's intermediate steps.

In [22]:
print_mcp_calls(product_search)


TAVILY MCP CALLS: Found 3 tool call(s):
--------------------------------------------------

Tool Call #1
   Type: mcp
   Tool Name: tavily_search
   Server: tavily
   Arguments: {'max_results': 10, 'query': 'iPhone models currently available on apple.com', 'search_depth': 'basic', 'top_k': 10, 'topic': 'general'}
   Could not parse tool data: Expecting value: line 1 column 1 (char 0)

Tool Call #2
   Type: mcp
   Tool Name: tavily_search
   Server: tavily
   Arguments: {'max_results': 10, 'query': 'iPhone models currently available on apple.com', 'search_depth': 'basic', 'topic': 'general'}
   Output: {'query': 'iPhone models currently available on apple.com', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.apple.com/in/iphone/compare/', 'title': 'Compare...

Tool Call #3
   Type: mcp
   Tool Name: tavily_extract
   Server: tavily
   Arguments: {'extract_depth': 'basic', 'format': 'markdown', 'urls': ['https://www.apple.com/in/iphone/compare

## Demo 3: Try it Yourself

Now it's your turn! Replace the query with some real time information you want to discover.


In [ ]:
your_query = "Your Query Here"  # Change this!

custom_response = connect_groq_to_tavily(client, tools, your_query)

In [ ]:
Markdown(custom_response["content"])

In [ ]:
print_mcp_calls(custom_response)